## 日内分时段的价量信息

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt

/home/aglv/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
clo = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/close_21_23.h5')
clo

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20210104 09:31        9.64     14.01      8.82     12.35      2.86     76.46   
         09:32        9.60     13.97      8.84     12.36      2.84     76.17   
         09:33        9.60     14.00      8.83     12.37      2.85     76.10   
         09:34        9.58     13.90      8.83     12.33      2.85     75.74   
         09:35        9.58     13.88      8.94     12.28      2.85     75.36   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56        7.13     10.87      5.78     19.02      2.81     37.33   
         14:57        7.13     10.87      5.79     19.04      2.82     37.33   
         14:58        7.13     10.87      5.79     19.04      2.82     37.33   
         14:59        7.13     10.87      5.79     19.04      2.82     37.33   
         15:00        7.14     10.88      5.79     18.98      2.81     37.33   

                  SH600010  SH600011  SH600012  SH600015  ...  SZ301519  \
Date     EndTime                                          ...             
20210104 09:31        1.18      4.49      6.12      6.22  ...       NaN   
         09:32        1.17      4.47      6.16      6.20  ...       NaN   
         09:33        1.17      4.47      6.17      6.21  ...       NaN   
         09:34        1.17      4.46      6.16      6.21  ...       NaN   
         09:35        1.17      4.47      6.16      6.19  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20231012 14:56        1.73      7.81     10.83      5.72  ...     23.05   
         14:57        1.73      7.82     10.82      5.73  ...     23.06   
         14:58        1.73      7.82     10.81      5.73  ...     23.06   
         14:59        1.73      7.82     10.81      5.73  ...     23.06   
         15:00        1.72      7.81     10.83      5.71  ...     23.05   

                  SZ301520  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56       65.80    101.99     84.05     44.24     41.43     61.38   
         14:57       65.65    101.81     84.08     44.28     41.39     61.37   
         14:58       65.64    101.81     84.10     44.28     41.39     61.34   
         14:59       65.64    101.81     84.10     44.28     41.39     61.34   
         15:00       65.64    101.90     84.29     44.35     41.40     61.33   

                  SZ301550  SZ301558  SZ301559  
Date     EndTime                                
20210104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20231012 14:56       50.40     14.08     22.72  
         14:57       50.21     14.01     22.69  
         14:58       50.18     14.00     22.69  
         14:59       50.18     14.00     22.69  
         15:00       50.30     14.01     22.69  

[161118 rows x 5175 columns]

In [8]:
clo.xs('20210104', level='Date').index[29]

'10:00'

In [3]:
min30_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[30, :] / x.iloc[0, :] - 1)
min30_ret

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.005187,0.009279,0.055556,-0.004049,0.000000,0.013471,-0.008475,0.000000,-0.006536,-0.004823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-0.010352,-0.011569,-0.073404,-0.002417,0.000000,0.001043,-0.008475,-0.013453,0.004950,-0.003231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,0.009346,0.016141,-0.017182,-0.001606,-0.003509,0.018951,0.000000,-0.004577,0.019386,0.003210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-0.010142,-0.014032,0.002415,-0.011146,0.000000,-0.007748,0.025862,0.000000,-0.011200,-0.007962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.001015,0.020360,-0.016867,0.001625,0.007143,0.007507,-0.008547,0.011364,0.004926,0.004808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,0.000000,-0.012422,0.012750,0.004122,0.003534,-0.003717,0.005848,-0.002532,-0.008101,-0.001745,...,0.007036,-0.051703,0.000000,0.002222,0.020377,0.006223,0.016644,0.019019,-0.022772,NaN
20231009,-0.009887,-0.022401,0.007143,-0.001034,0.003534,-0.019271,0.000000,0.001271,-0.010830,-0.008787,...,-0.007420,-0.006976,-0.005789,-0.002612,-0.007039,-0.018608,-0.015832,0.006105,-0.079412,NaN
20231010,-0.011220,0.005561,-0.001757,-0.012487,0.000000,0.005650,-0.005780,-0.023899,-0.006323,0.000000,...,0.000000,-0.039312,-0.023664,0.015869,0.009888,-0.003998,-0.003778,0.072708,0.103571,NaN


In [10]:
min30_toend_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[-1, :] / x.iloc[30, :] - 1)
min30_toend_ret

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,0.010428,-0.007072,-0.009667,0.008130,0.000000,0.000000,0.008547,0.002227,-0.003289,0.003231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,0.012552,0.010241,0.019518,0.014540,0.003509,0.006904,0.000000,0.000000,0.001642,0.014587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,0.010288,-0.022383,-0.011655,0.020113,0.000000,-0.012866,-0.008547,0.018391,0.006339,0.003200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,0.005123,-0.038951,0.015663,-0.000805,-0.007092,-0.012884,-0.016807,0.004566,-0.012945,0.001605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.001016,0.011512,0.004902,0.005677,0.024823,-0.006797,0.008621,0.004494,-0.004902,-0.003190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.006993,0.006289,0.000000,-0.006157,0.000000,0.009861,0.000000,-0.001269,0.007260,0.000000,...,-0.002877,-0.044331,-0.033232,-0.010736,0.008465,-0.020156,-0.019050,-0.012301,-0.040020,NaN
20231009,0.002853,-0.010999,0.001773,-0.006725,-0.007042,-0.002153,0.000000,-0.001269,0.013686,0.007092,...,-0.015365,0.003994,-0.005071,-0.017617,-0.016327,-0.005384,-0.015623,0.024482,-0.070288,NaN
20231010,-0.005674,-0.001843,0.007042,-0.000527,-0.003546,-0.007223,-0.005814,-0.006443,-0.010909,-0.001767,...,-0.021932,-0.037108,-0.000097,-0.010255,-0.029373,-0.019362,-0.021646,-0.028161,-0.063430,NaN


In [12]:
sym = min30_ret * min30_toend_ret
sym = sym / abs(sym)
sym

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-1.0,-1.0,-1.0,-1.0,NaN,NaN,-1.0,NaN,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-1.0,-1.0,-1.0,-1.0,NaN,1.0,NaN,NaN,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,1.0,-1.0,1.0,-1.0,NaN,-1.0,NaN,-1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-1.0,1.0,1.0,1.0,NaN,1.0,-1.0,NaN,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,NaN,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,-1.0,NaN,...,-1.0,1.0,NaN,-1.0,1.0,-1.0,-1.0,-1.0,1.0,NaN
20231009,-1.0,1.0,1.0,1.0,-1.0,1.0,NaN,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
20231010,1.0,-1.0,-1.0,1.0,NaN,-1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,NaN


In [19]:
sym.isna().sum(axis=1)

Date
20210104    1480
20210105    1418
20210106    1381
20210107    1248
20210108    1410
            ... 
20230928     691
20231009     537
20231010     596
20231011     632
20231012     598
Length: 671, dtype: int64

In [28]:
def inverse(p:pd.DataFrame, l_min=30):
    l_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[l_min, :] / x.iloc[0, :] - 1)
    r_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[-1, :] / x.iloc[l_min, :] -1)

    sym = l_ret * r_ret
    sym = sym

    num = sym.shape[1] - sym.isna().sum(axis=1)
    inv_per = sym.sum(axis=1) / num
    
    return inv_per, sym, num

In [29]:
inv_per_30, sym_30, len_30 = inverse(clo, 30)

In [31]:
sym_30.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/min30_reverse_multi.csv')

In [7]:
inv_per_30[inv_per_30 > 0.3]

Date
20210107    0.491215
20210111    0.320453
20210129    0.375773
20210219    0.552888
20210310    0.422368
20210416    0.317719
20210924    0.356791
20210927    0.516331
20211027    0.350175
20211206    0.319962
20211224    0.369097
20220120    0.502164
20220125    0.600356
20220127    0.520623
20220208    0.339150
20220308    0.627451
20220309    0.378816
20220315    0.665788
20220318    0.332567
20220426    0.592132
20220510    0.435093
20220524    0.435498
20220609    0.422764
20220610    0.406558
20220629    0.388727
20220705    0.300846
20220712    0.310615
20220802    0.528848
20220824    0.506831
20220829    0.324503
20220831    0.541223
20220915    0.515165
20220923    0.331471
20220926    0.366645
20220928    0.335539
20220930    0.301543
20221028    0.554135
20221111    0.341587
20221115    0.379590
20221219    0.563690
20221222    0.418221
20221226    0.301883
20230209    0.406052
20230321    0.323805
20230425    0.454194
20230428    0.368739
20230606    0.386799
20230627

In [32]:
sym_30[inv_per_30 > 0.1]

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210106,1.0,-1.0,1.0,-1.0,NaN,-1.0,NaN,-1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-1.0,1.0,1.0,1.0,NaN,1.0,-1.0,NaN,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210111,-1.0,1.0,-1.0,-1.0,NaN,1.0,-1.0,1.0,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210113,1.0,1.0,1.0,-1.0,-1.0,1.0,NaN,-1.0,NaN,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210122,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230918,NaN,-1.0,1.0,-1.0,NaN,-1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,-1.0,-1.0,-1.0,1.0,NaN,-1.0,NaN,NaN
20230919,-1.0,-1.0,-1.0,1.0,NaN,-1.0,-1.0,-1.0,NaN,1.0,...,1.0,NaN,1.0,-1.0,-1.0,1.0,NaN,-1.0,NaN,NaN
20230922,NaN,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,NaN,-1.0,...,-1.0,NaN,1.0,1.0,1.0,1.0,-1.0,1.0,NaN,NaN


In [8]:
eg = clo.xs('20210107', level='Date')
eg2 = eg.iloc[-1, :] / eg.iloc[0, :] - 1
eg2.mean()

-0.021180365723860194

In [9]:
index_close = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/data_index/close.csv')
hs300_clo = index_close.iloc[2, 1:]
hs300_clo.index = [str(i) for i in hs300_clo.index]
hs300_clo.index.names = ['date']
hs300_clo = hs300_clo.loc['20210101':]
hs300_clo

date
20210104    5267.7181
20210105    5368.5049
20210106    5417.6677
20210107    5513.6568
20210108    5495.4306
              ...    
20231107    3619.7593
20231108    3611.0723
20231109    3612.8263
20231110    3586.4893
20231113    3579.4141
Name: 2, Length: 693, dtype: float64

In [12]:
index_op = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/data_index/open.csv')
hs300_op = index_op.iloc[2, 1:]
hs300_op.index = [str(i) for i in hs300_op.index]
hs300_op.index.names = ['date']
hs300_op = hs300_op.loc['20210101':]
hs300_op

date
20210104    5212.9313
20210105    5245.8355
20210106    5386.5144
20210107    5428.3944
20210108    5526.2876
              ...    
20231107    3623.7849
20231108    3610.6434
20231109    3603.1082
20231110    3596.0688
20231113    3593.9109
Name: 2, Length: 693, dtype: float64

In [27]:
# for date in inv_per_30[inv_per_30 > 0.4].index:
#     # 创建新的图形和坐标轴
#     fig, ax = plt.subplots(figsize=(10, 5))
    
#     date_id = hs300_clo.index.get_loc(date)
#     dates = hs300_clo.index[date_id-2:date_id+3]  # 取前后两天

#     # 确保绘制的数据与dates匹配
#     for i in range(len(dates)):
#         op = hs300_op[dates[i]]
#         cl = hs300_clo[dates[i]]

#         if cl >= op:
#             color = 'red'
#             ax.plot([dates[i], dates[i]], [op, cl], color=color, linewidth=4)
#         else:
#             color = 'green'
#             ax.plot([dates[i], dates[i]], [cl, op], color=color, linewidth=4)

#     # 设置X轴标签和标题
#     # ax.set_xticks(dates)
#     # ax.set_xticklabels(dates.date, rotation=45)
#     # ax.set_title(f'K线图示例 - {date.date()}')
#     # ax.set_ylabel('价格')

#     plt.tight_layout()
#     plt.show()
